## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

July 21 July 27


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,July-23-2025,Condo/Co-op,325 S Biscayne Blvd #1823,Miami,FL,33131.0,1000000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/325-S-Biscayne...,MARMLS,A11838781,N,Y,25.771705,-80.186373


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-25    31
2025-07-23    20
2025-07-22    20
2025-07-24    17
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
3,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.877870,-80.121437
102,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-1003,Surfside,FL,33154.0,18350000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11650756,N,Y,25.877870,-80.121437
84,PAST SALE,2025-07-25,Condo/Co-op,18501 Collins Ave #2103,Sunny Isles Beach,FL,33160.0,5050000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11734072,N,Y,25.947720,-80.120228
64,PAST SALE,2025-07-22,Condo/Co-op,9999 Collins Ave Unit 11A,Bal Harbour,FL,33154.0,3400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11770369,N,Y,25.891991,-80.122782
58,PAST SALE,2025-07-24,Condo/Co-op,100 S Pointe Dr #1803,Miami Beach,FL,33139.0,3275000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11795185,N,Y,25.767837,-80.132783
69,PAST SALE,2025-07-25,Condo/Co-op,3301 NE 183rd St #2807,Aventura,FL,33160.0,1790000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11782191,N,Y,25.946197,-80.137058
49,PAST SALE,2025-07-25,Condo/Co-op,6365 Collins Ave #4407,Miami Beach,FL,33141.0,1787000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11742690,N,Y,25.846069,-80.119697
109,PAST SALE,2025-07-22,Condo/Co-op,1111 Crandon Blvd Unit A201,Key Biscayne,FL,33149.0,1725000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11624683,N,Y,25.684030,-80.157650
68,PAST SALE,2025-07-25,Condo/Co-op,6103 Aqua Ave #401,Miami Beach,FL,33141.0,1590000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6103-Aqu...,MARMLS,A11773792,N,Y,25.843207,-80.122891
91,PAST SALE,2025-07-25,Condo/Co-op,15811 Collins Ave #1603,Sunny Isles Beach,FL,33160.0,1550000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11731295,N,Y,25.922834,-80.121864


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

88


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,259,057


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$635


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$110,797,028


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
85
-----------
Input Previous Week Condo Average Sales Price:
1049321
-----------
Input Previous Week Condo Average PSF:
607
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
89192316.0


In [25]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,85,1049321,607,89192316.0


In [26]:
last_week['Last Week Total Dollar Volume'].iloc[0]

89192316.0

In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           88             1259057          635                    110797028.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.87787,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange
1,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-1003,Surfside,FL,33154.0,18350000.0,4.0,5.5,SURF CLUB CONDO,4235.0,NaN,2017.0,NaN,4333.0,20183.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11650756,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,None,None,None,None,None,None,None,None,2,blue
2,PAST SALE,2025-07-25,Condo/Co-op,18501 Collins Ave #2103,Sunny Isles Beach,FL,33160.0,5050000.0,4.0,4.5,NORTH BISCAYNE BEACH AMEN,2797.0,NaN,2020.0,NaN,1806.0,5376.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11734072,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Sergio Lopez De Mesa,Vanguard Properties,None,None,Rita Collins,Douglas Elliman,None,None,3,blue
3,PAST SALE,2025-07-22,Condo/Co-op,9999 Collins Ave Unit 11A,Bal Harbour,FL,33154.0,3400000.0,3.0,4.5,BAL HARBOUR TOWER CONDO,3670.0,NaN,1990.0,NaN,926.0,5676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11770369,N,Y,25.891991,-80.122782,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",Abe Ash,"Multivest Realty, Inc.",None,None,4,blue
4,PAST SALE,2025-07-24,Condo/Co-op,100 S Pointe Dr #1803,Miami Beach,FL,33139.0,3275000.0,1.0,2.0,CONTINUUM ON SOUTH BEACH,1365.0,NaN,2002.0,NaN,2399.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11795185,N,Y,25.767837,-80.132783,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2025-07-25,Condo/Co-op,3301 NE 183rd St #2807,Aventura,FL,33160.0,1790000.0,3.0,3.5,THE PENINSULA II CONDO,3069.0,NaN,2007.0,NaN,583.0,2915.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11782191,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Carlota Fonseca,Smart Luxe,None,None,Sonia Darwich Israel,"Related ISG Realty, LLC.",None,None,6,blue
6,PAST SALE,2025-07-25,Condo/Co-op,6365 Collins Ave #4407,Miami Beach,FL,33141.0,1787000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1352.0,1815.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11742690,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Nabil Jimenez,Douglas Elliman,None,None,Andrea Trujillo,EXP Realty LLC,None,None,7,blue
7,PAST SALE,2025-07-22,Condo/Co-op,1111 Crandon Blvd Unit A201,Key Biscayne,FL,33149.0,1725000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,968.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11624683,N,Y,25.684030,-80.157650,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,Maureen Jauregui,Coldwell Banker Realty,Emma Jauregui,Coldwell Banker Realty,Juan Velasquez,Metrocoma Realty,None,None,8,blue
8,PAST SALE,2025-07-25,Condo/Co-op,6103 Aqua Ave #401,Miami Beach,FL,33141.0,1590000.0,3.0,3.5,SPEAR AT AQUA CONDO,2203.0,NaN,2004.0,NaN,722.0,3992.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6103-Aqu...,MARMLS,A11773792,N,Y,25.843207,-80.122891,https://www.redfin.com/FL/Miami-Bea

In [35]:
%store -r map_box_api_key

In [36]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.87787,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_07282025


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'SURF CLUB CONDO at 9001 Collins Ave Unit S-PH2 in Surfside'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-PH2...
1,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-1003,Surfside,FL,33154.0,18350000.0,4.0,5.5,SURF CLUB CONDO,4235.0,NaN,2017.0,NaN,4333.0,20183.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11650756,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,None,None,None,None,None,None,None,None,2,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
2,PAST SALE,2025-07-25,Condo/Co-op,18501 Collins Ave #2103,Sunny Isles Beach,FL,33160.0,5050000.0,4.0,4.5,NORTH BISCAYNE BEACH AMEN,2797.0,NaN,2020.0,NaN,1806.0,5376.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11734072,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Sergio Lopez De Mesa,Vanguard Properties,None,None,Rita Collins,Douglas Elliman,None,None,3,blue,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
3,PAST SALE,2025-07-22,Condo/Co-op,9999 Collins Ave Unit 11A,Bal Harbour,FL,33154.0,3400000.0,3.0,4.5,BAL HARBOUR TOWER CONDO,3670.0,NaN,1990.0,NaN,926.0,5676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11770369,N,Y,25.891991,-80.122782,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",Abe Ash,"Multivest Realty, Inc.",None,None,4,blue,BAL HARBOUR TOWER CONDO at 9999 Collins Ave Un...
4,PAST SALE,2025-07-24,Condo/Co-op,100 S Pointe Dr #1803,Miami Beach,FL,33139.0,3275000.0,1.0,2.0,CONTINUUM ON SOUTH BEACH,1365.0,NaN,2002.0,NaN,2399.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11795185,N,Y,25.767837,-80.132783,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,5,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
5,PAST SALE,2025-07-25,Condo/Co-op,3301 NE 183rd St #2807,Aventura,FL,33160.0,1790000.0,3.0,3.5,THE PENINSULA II CONDO,3069.0,NaN,2007.0,NaN,583.0,2915.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11782191,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Carlota Fonseca,Smart Luxe,None,None,Sonia Darwich Israel,"Related ISG Realty, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #28...
6,PAST SALE,2025-07-25,Condo/Co-op,6365 Collins Ave #4407,Miami Beach,FL,33141.0,1787000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1352.0,1815.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11742690,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Nabil Jimenez,Douglas Elliman,None,None,Andrea Trujillo,EXP Realty LLC,None,None,7,blue,AKOYA CONDO at 6365 Collins Ave #4407 in Miami...
7,PAST SALE,2025-07-22,Condo/Co-op,1111 Crandon Blvd Unit A201,Key Biscayne,FL,33149.0,1725000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,968.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11624683,N,Y,25.684030,-80.157650,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,Maureen Jauregui,Coldwell Banker Realty,Emma Jauregui,Co

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,550,000 to $38,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Surfside, Miami Beach, Sunny Isles Beach, Key Biscayne, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 88 condo sales totaling $110,797,028 million from July 21 to July 27. The previous week, brokers closed 85 condo sales totaling $89,192,316.

Last week’s units sold for an average of $1,259,057, higher than the $1,049,321 

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-PH2...
1,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-1003,Surfside,FL,33154.0,18350000.0,4.0,5.5,SURF CLUB CONDO,4235.0,NaN,2017.0,NaN,4333.0,20183.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11650756,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,None,None,None,None,None,None,None,None,2,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
2,PAST SALE,2025-07-25,Condo/Co-op,18501 Collins Ave #2103,Sunny Isles Beach,FL,33160.0,5050000.0,4.0,4.5,NORTH BISCAYNE BEACH AMEN,2797.0,NaN,2020.0,NaN,1806.0,5376.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11734072,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Sergio Lopez De Mesa,Vanguard Properties,None,None,Rita Collins,Douglas Elliman,None,None,3,blue,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
3,PAST SALE,2025-07-22,Condo/Co-op,9999 Collins Ave Unit 11A,Bal Harbour,FL,33154.0,3400000.0,3.0,4.5,BAL HARBOUR TOWER CONDO,3670.0,NaN,1990.0,NaN,926.0,5676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11770369,N,Y,25.891991,-80.122782,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",Abe Ash,"Multivest Realty, Inc.",None,None,4,blue,BAL HARBOUR TOWER CONDO at 9999 Collins Ave Un...
4,PAST SALE,2025-07-24,Condo/Co-op,100 S Pointe Dr #1803,Miami Beach,FL,33139.0,3275000.0,1.0,2.0,CONTINUUM ON SOUTH BEACH,1365.0,NaN,2002.0,NaN,2399.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11795185,N,Y,25.767837,-80.132783,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,5,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
5,PAST SALE,2025-07-25,Condo/Co-op,3301 NE 183rd St #2807,Aventura,FL,33160.0,1790000.0,3.0,3.5,THE PENINSULA II CONDO,3069.0,NaN,2007.0,NaN,583.0,2915.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11782191,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Carlota Fonseca,Smart Luxe,None,None,Sonia Darwich Israel,"Related ISG Realty, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #28...
6,PAST SALE,2025-07-25,Condo/Co-op,6365 Collins Ave #4407,Miami Beach,FL,33141.0,1787000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1352.0,1815.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11742690,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Nabil Jimenez,Douglas Elliman,None,None,Andrea Trujillo,EXP Realty LLC,None,None,7,blue,AKOYA CONDO at 6365 Collins Ave #4407 in Miami...
7,PAST SALE,2025-07-22,Condo/Co-op,1111 Crandon Blvd Unit A201,Key Biscayne,FL,33149.0,1725000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,968.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11624683,N,Y,25.684030,-80.157650,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,Maureen Jauregui,Coldwell Banker Realty,Emma Jauregui,Co

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,550,000 to $38,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Surfside, Miami Beach, Sunny Isles Beach, Key Biscayne, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 88 condo sales totaling $110,797,028 million from July 21 to July 27. The previous week, brokers closed 85 condo sales totaling $89,192,316.

Last week’s units sold for an average of $1,259,057, higher than the $1,049,321 

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S-PH2/home/196451384


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S1003/home/79133358


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/North-Miami-Beach/15811-Collins-Ave-33160/unit-1603/home/43446211


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/6103-Aqua-Ave-33141/unit-401/home/43310914


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S-PH2/home/196451384


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/3301-NE-183rd-St-33160/unit-2807/home/43397057


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-PH2,Surfside,FL,33154.0,38200000.0,5.0,8.5,SURF CLUB CONDO,5675.0,NaN,2017.0,NaN,6731.0,22513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11841922,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-PH2...
1,PAST SALE,2025-07-25,Condo/Co-op,9001 Collins Ave Unit S-1003,Surfside,FL,33154.0,18350000.0,4.0,5.5,SURF CLUB CONDO,4235.0,NaN,2017.0,NaN,4333.0,20183.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11650756,N,Y,25.877870,-80.121437,https://www.redfin.com/FL/Surfside/9001-Collin...,None,None,None,None,None,None,None,None,2,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
2,PAST SALE,2025-07-25,Condo/Co-op,18501 Collins Ave #2103,Sunny Isles Beach,FL,33160.0,5050000.0,4.0,4.5,NORTH BISCAYNE BEACH AMEN,2797.0,NaN,2020.0,NaN,1806.0,5376.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11734072,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Sergio Lopez De Mesa,Vanguard Properties,None,None,Rita Collins,Douglas Elliman,None,None,3,blue,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
3,PAST SALE,2025-07-22,Condo/Co-op,9999 Collins Ave Unit 11A,Bal Harbour,FL,33154.0,3400000.0,3.0,4.5,BAL HARBOUR TOWER CONDO,3670.0,NaN,1990.0,NaN,926.0,5676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,MARMLS,A11770369,N,Y,25.891991,-80.122782,https://www.redfin.com/FL/Bal-Harbour/9999-Col...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",Abe Ash,"Multivest Realty, Inc.",None,None,4,blue,BAL HARBOUR TOWER CONDO at 9999 Collins Ave Un...
4,PAST SALE,2025-07-24,Condo/Co-op,100 S Pointe Dr #1803,Miami Beach,FL,33139.0,3275000.0,1.0,2.0,CONTINUUM ON SOUTH BEACH,1365.0,NaN,2002.0,NaN,2399.0,3600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11795185,N,Y,25.767837,-80.132783,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,None,None,None,None,None,None,None,None,5,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #1...
5,PAST SALE,2025-07-25,Condo/Co-op,3301 NE 183rd St #2807,Aventura,FL,33160.0,1790000.0,3.0,3.5,THE PENINSULA II CONDO,3069.0,NaN,2007.0,NaN,583.0,2915.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3301-NE-183...,MARMLS,A11782191,N,Y,25.946197,-80.137058,https://www.redfin.com/FL/Aventura/3301-NE-183...,Carlota Fonseca,Smart Luxe,None,None,Sonia Darwich Israel,"Related ISG Realty, LLC.",None,None,6,blue,THE PENINSULA II CONDO at 3301 NE 183rd St #28...
6,PAST SALE,2025-07-25,Condo/Co-op,6365 Collins Ave #4407,Miami Beach,FL,33141.0,1787000.0,2.0,2.0,AKOYA CONDO,1322.0,NaN,2004.0,NaN,1352.0,1815.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6365-Col...,MARMLS,A11742690,N,Y,25.846069,-80.119697,https://www.redfin.com/FL/Miami-Beach/6365-Col...,Nabil Jimenez,Douglas Elliman,None,None,Andrea Trujillo,EXP Realty LLC,None,None,7,blue,AKOYA CONDO at 6365 Collins Ave #4407 in Miami...
7,PAST SALE,2025-07-22,Condo/Co-op,1111 Crandon Blvd Unit A201,Key Biscayne,FL,33149.0,1725000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,968.0,2291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11624683,N,Y,25.684030,-80.157650,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,Maureen Jauregui,Coldwell Banker Realty,Emma Jauregui,Co

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 16) ## List (Earlier) date
date2 = datetime(2025, 7, 25) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

312


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [71]:
chart_df['PPSF'].astype(float)

0    6731.0
1    4333.0
2    1806.0
3     926.0
4    2399.0
5     583.0
6    1352.0
7     968.0
8     722.0
9     801.0
Name: PPSF, dtype: float64

In [72]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [73]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [74]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-25,SURF CLUB CONDO,9001 Collins Ave Unit S-PH2 in Surfside,"$38,200,000","5,675","$6,731",2017,"Ximena Penuela Fort Realty, LLC","Ximena Penuela Fort Realty, LLC"
1,2025-07-25,SURF CLUB CONDO,9001 Collins Ave Unit S-1003 in Surfside,"$18,350,000","4,235","$4,333",2017,,
2,2025-07-25,NORTH BISCAYNE BEACH AMEN,18501 Collins Ave #2103 in Sunny Isles Beach,"$5,050,000","2,797","$1,806",2020,Sergio Lopez De Mesa Vanguard Properties,Rita Collins Douglas Elliman
3,2025-07-22,BAL HARBOUR TOWER CONDO,9999 Collins Ave Unit 11A in Bal Harbour,"$3,400,000","3,670",$926,1990,"Marcello Agostini AG Real Estate Advisors, LLC...","Abe Ash Multivest Realty, Inc."
4,2025-07-24,CONTINUUM ON SOUTH BEACH,100 S Pointe Dr #1803 in Miami Beach,"$3,275,000","1,365","$2,399",2002,,
5,2025-07-25,THE PENINSULA II CONDO,3301 NE 183rd St #2807 in Aventura,"$1,790,000","3,069",$583,2007,Carlota Fonseca Smart Luxe,"Sonia Darwich Israel Related ISG Realty, LLC."
6,2025-07-25,AKOYA CONDO,6365 Collins Ave #4407 in Miami Beach,"$1,787,000","1,322","$1,352",2004,Nabil Jimenez Douglas Elliman,Andrea Trujillo EXP Realty LLC
7,2025-07-22,TOWERS OF KEY BISCAYNE CO,1111 Crandon Blvd Unit A201 in Key Biscayne,"$1,725,000","1,782",$968,1972,Maureen Jauregui Coldwell Banker Realty Emma J...,Juan Velasquez Metrocoma Realty
8,2025-07-25,SPEAR AT AQUA CONDO,6103 Aqua Ave #401 in Miami Beach,"$1,590,000","2,203",$722,2004,"Nancy Batchelor Compass Florida, LLC. Simone W...","Nancy Batchelor Compass Florida, LLC. Simone W..."
9,2025-07-25,TDR TOWER III CONDO,15811 Collins Ave #1603 in Sunny Isles Beach,"$1,550,000","1,935",$801,2010,Patricia Gomez PFS Realty LLC Keilly Sanchez P...,Christopher Borchetta PA Luxe Properties


In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [77]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')